In [1]:
import sys
sys.path.append('./../src')

In [285]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import re
from importlib import reload
from pymongo import MongoClient
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

## Downloading Data from MongoDB ##

In [171]:
with open('../pickles/raw_gd_corpus.pickle', 'wb') as to_write:
    pickle.dump(gd_corpus, to_write)

In [172]:
with open('../pickles/raw_g3s_corpus.pickle', 'wb') as to_write:
    pickle.dump(gd_corpus, to_write)

In [184]:
with open('../pickles/raw_combined_corpus.pickle', 'wb') as to_write:
    pickle.dump(combined_corpus, to_write)

In [69]:
client = MongoClient()
db = client.blogs
db.list_collection_names()

['g3s', 'gd']

In [70]:
cursor = db.gd.find({}, {'_id': 0, 'text': 1})

In [71]:
# Store all gd texts (queried from mongodb) in a list of strings 
gd_corpus = [result['text'] for result in cursor]    

In [72]:
cursor = db.g3s.find({}, {'_id': 0, 'text': 1})

In [73]:
# Store all gd texts (queried from mongodb) in a list of strings 
g3s_corpus = [result['text'] for result in cursor]  

In [173]:
g3s_corpus[150]

'\n\nFollower Fridays is a series of profiles highlighting members of Gaysian Third Space to showcase the diversity of gaysians in the Community. This week’s featured member is @beat0t.\n\n\n\nWho are you?\n\n\n\nMy name is Bea. I’m a recent college graduate, co-founder of a dance company based in Philadelphia called D2D: Dare To Dance. \n\n\n\nWhere are you from?\n\n\n\nI was born and raised in the island of Leyte, Philippines. Lived there most of my life before moving to the States when I was 12.\n\n\n\nWhat do you do?\n\n\n\nI’m you’re typical post grad, looking to jump start their career. I also teach dance at a local dance studio in Philadelphia\n\n\n\nWhat are you passionate about?\n\n\n\nI truly love dancing and I want to use dance as an avenue to change the world, no matter how small that change may be. I am also passionate about my Pilipinx identity. Little by little, I’m learning more and more about being a Pilipinx, I fall more and more in love about my heritage and my ident

In [75]:
count_follower_fridays = 0
for text in g3s_corpus:
    if 'Follower Fridays' in text:
        count_follower_fridays += 1
count_follower_fridays

139

In [76]:
count_come_out = 0
for text in g3s_corpus:
    if 'come out' in text:
        count_come_out += 1
count_come_out

18

In [77]:
count_coming_out = 0
for text in g3s_corpus:
    if 'coming out' in text:
        count_coming_out += 1
count_coming_out

20

In [124]:
count_coming_out = 0
for text in gd_corpus:
    if 'coming out' in text:
        count_coming_out += 1
count_coming_out

6

In [181]:
combined_corpus = gd_corpus + g3s_corpus

In [183]:
combined_corpus[33]

'Dear Diary,\n\nI spend a lot of time analyzing how much our past experiences affect who we are today. Growing up, my father wasn’t around much. He did business internationally while my mom took care of us at home. My parents were always supportive of me and my sisters. My dad was endearing but was never a man of many words. My mother cared for us so much and always wanted the best for us. Every night before bed, my mom would hold my hands and tell me she loved me. She made sure I knew that every night before I fell asleep.\n\nWhen I was 8, my mom found out my father was having an affair during his Asia business trips. I don’t know how she found out, but she somehow obtained a photo of the mistress and some letters that my dad had written to her.\xa0I noticed my mom staying up late at night crying, obsessively staring at the evidence that my father was being unfaithful.\xa0\n\nMy mom let us know what my dad was doing and told us it was not ok. It hurt us so much to see our mom crying e

## Text Preprocessing ##

In [147]:
# Removed 'out' from sklearn stop_words list since 'coming out' is an important term in LGBT texts
from sklearn.feature_extraction import text
custom_stop_words = text.ENGLISH_STOP_WORDS.difference({'out'})

In [185]:
# Remove numbers from corpus
clean_combined_corpus = [re.sub('\w*\d+\w*', '', text) for text in combined_corpus]

In [187]:
clean_combined_corpus[28]

'Dear Diary,\n\nI feel like I’ve been out forever, but it hasn’t been that long. When I started writing this entry on\xa0April , , I had come out exactly three years ago.\n\nI was nineteen, a sophomore in college, when I came out to my first friend: he was a good\xa0college friend of mine who, like me, was gay and Vietnamese American. It was a Tuesday night when it happened.\xa0I had asked my friend earlier if we could talk at a quiet café on the edge of campus. I didn’t mention to him what it was about. As I walked to the café, I clutched an index\xa0card inside my jacket’s pocket as if everything depended on it.\n\nDays before, I had been deliberating about how I should come out to my friend. I was afraid\xa0that I wouldn’t be able to say the words to him when the time came, so I compromised: I’ll\xa0write “I’m gay” on an index card and give it to him—simple as that. But when I wrote the index\xa0card the night before, I decided I didn’t want to just write “I’m gay”. Instead, I wrote

In [191]:
nlp = spacy.load('en_core_web_sm')
nlp.Defaults.stop_words.remove("out")

In [194]:
nlp.Defaults.stop_words

False

In [197]:
docs = list(nlp.pipe(clean_combined_corpus))

In [204]:
type(docs[0])

spacy.tokens.doc.Doc

In [236]:
# Lemmatize all the docs so I get all words reduced to their dictionary (lemma) forms
lemmatized_docs = []
for doc in docs:
    # Don't lemmatize the word if it's a pronoun
    lemmatized_docs.append(' '.join([token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]))

In [239]:
print(lemmatized_docs[100])

thoughtsfromthewalkhome : 

 in one of my course last week , we discuss how social network affect a person ’s health status . that be to say , how access to resource , the ability to obtain emotional support , and a sense of belong – or the lack ofthese benefit result from a poor social network – can influence howhealthy a person be , both physically and mentally . at the end of the lecture , 
 the professor show part 
 of a tedtalk about a composer who create a virtual choir by have people 
 submit YouTube video and stitch together the result . the video have a 
 quotation that really strike me . 

 It strike me because of how often my professor , at the 
 close of class , keep qualify the kind of connection about which the woman 
 write as a " virtual " one . to be sure , this extra adjective be add for the 
 sake of instruction . however , it remind me of how often and unconsciously many 
 of us distinguish between what happen online and in 
 the " real world " . what we do online s

## Testing CountVectorizer and TfidfVectorizer ##

In [275]:
cv = CountVectorizer(stop_words=custom_stop_words, ngram_range=(1, 2), min_df=3)
cv_X = cv.fit_transform(lemmatized_docs).toarray()

In [276]:
tfidf = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(1, 2), min_df=3)
tfidf_X = tfidf.fit_transform(lemmatized_docs).toarray()

In [277]:
cv_X.shape
tfidf_X.shape

(376, 5448)

(376, 5448)

In [278]:
cv_doc_term = pd.DataFrame(cv_X, columns=cv.get_feature_names())
tfidf_doc_term = pd.DataFrame(tfidf_X, columns=tfidf.get_feature_names())

In [280]:
cv_doc_term.head()

,aapi,abandon,abc,ability,able,able make,able meet,able travel,able work,abroad,abs,absence,absolute,absolutely,abuse,academia,academic,accent,accept,accept love,accept open,accept sexuality,acceptable,acceptance,access,accommodate,accompany,accomplish,accomplishment,accord,account,accountability,accountable,accumulate,accurately,accuse,achieve,achievement,acknowledge,acquaintance,act,act like,act way,acting,action,action speak,active,actively,activism,activist,activity,actor,actual,actually,actually like,actually quite,actually talk,actually tell,actually want,acutely,ad,adam,adapt,add,addict,addiction,addition,additional,address,address issue,adequate,adhere,adjust,admire,admission,admit,adolescent,adopt,adore,adult,adulthood,advantage,adventure,advertising,advice,advise,advisor,advocacy,advocate,aesthetic,affair,affect,affect community,affection,affectionate,affirm,affirmation,afford,affordable,afraid,african,african american,afterlife,afternoon,afterward,age,age gay,agency,agenda,aggression,ago,agree,agree disagree,aha,ahead,aids,aim,air,airport,aisle,aka,akin,alarm,albeit,alcohol,alex,align,alive,alliance,allow,allow share,allow travel,ally,alostfish,alternate,alternative,altogether,amateur,amazing,amazing friend,ambition,america,american,american bear,american community,american culture,american family,american gay,american grow,american history,american male,american man,american student,americanize,americans,amusement,analysis,analyst,analyze,ancestor,ancestral,ancient,andrew,andy,anecdote,angel,angeles,anger,angle,angry,anguish,animal,animation,anime,anniversary,announce,annoyed,annoying,annual,anonymous,answer,answer question,antagonize,anthony,anti,anticipation,antonio,anxiety,anxious,anybody,anymore,anymore know,anyways,apart,apartment,api,api community,api equality,apologetic,apologize,apology,app,app like,apparently,appeal,appear,appearance,apple,application,apply,appreciate,appreciation,approach,approach date,appropriate,approve,april,architect,architecture,arduous,area,area passionate,arguably,argue,argument,arise,arm,army,arrange,arrangement,arrival,arrive,art,art form,art passionate,article,articulate,artist,artistic,ashamed,ashamed sexuality,asia,asian,asian america,asian american,asian americans,asian asian,asian community,asian country,asian culture,asian date,asian ethnicity,asian friend,asian gay,asian guy,asian identity,asian male,asian man,asian pacific,asian parent,asian people,asian pride,asian student,asian white,asians,asians asian,asians date,aside,ask,ask boyfriend,ask gay,ask help,ask know,ask like,ask mother,ask question,ask want,asleep,aspect,aspiration,aspire,ass,asshole,assimilation,assistant,associate,assume,assumption,assure,atheist,attach,attain,attempt,attend,attendant,attention,attitude,attorney,attract,attraction,attractive,attractive asian,attractiveness,attribute,audience,aunt,australia,australian,authentic,authenticity,author,automatically,available,avenue,average,avid,avoid,await,awake,award,aware,awareness,away,away want,awesome,awful,awhile,awkward,ba,baby,bachelor,background,backpack,backwards,bad,bad case,bad thing,bad way,badly,badly want,bag,baggage,bake,baker,bakery,baking,balance,ball,bamboo,band,bangkok,bank,banter,bar,bare,barely,barista,barrier,base,basic,basically,basis,bathroom,battle,bay,bay area,beach,bear,bear chinese,bear raise,beat,beautiful,beautiful thing,beautifully,beauty,bed,bed night,beg,begin,begin explore,beginning,behave,behavior,beijing,belief,believe,believe deserve,believe good,believe love,believe make,believe parent,believe people,believe world,believer,belong,belonging,belt,beneath,beneficial,benefit,berkeley,bet,better,betterment,bi,bias,bible,big,big deal,big picture,big thank,bigotry,bike,bilingual,billion,binary,bind,binge,binge watch,biological,biology,birth,birthday,bisexual,bit,bit piece,bitch,bite,bitter,bitter taste,bitterness,black,black white,blah,blame,blanket,blast,blatant,blatant racism,bleed,blend,bless,blessing,blind,block,blog,blogg

In [281]:
tfidf_doc_term.head()

,aapi,abandon,abc,ability,able,able make,able meet,able travel,able work,abroad,abs,absence,absolute,absolutely,abuse,academia,academic,accent,accept,accept love,accept open,accept sexuality,acceptable,acceptance,access,accommodate,accompany,accomplish,accomplishment,accord,account,accountability,accountable,accumulate,accurately,accuse,achieve,achievement,acknowledge,acquaintance,act,act like,act way,acting,action,action speak,active,actively,activism,activist,activity,actor,actual,actually,actually like,actually quite,actually talk,actually tell,actually want,acutely,ad,adam,adapt,add,addict,addiction,addition,additional,address,address issue,adequate,adhere,adjust,admire,admission,admit,adolescent,adopt,adore,adult,adulthood,advantage,adventure,advertising,advice,advise,advisor,advocacy,advocate,aesthetic,affair,affect,affect community,affection,affectionate,affirm,affirmation,afford,affordable,afraid,african,african american,afterlife,afternoon,afterward,age,age gay,agency,agenda,aggression,ago,agree,agree disagree,aha,ahead,aids,aim,air,airport,aisle,aka,akin,alarm,albeit,alcohol,alex,align,alive,alliance,allow,allow share,allow travel,ally,alostfish,alternate,alternative,altogether,amateur,amazing,amazing friend,ambition,america,american,american bear,american community,american culture,american family,american gay,american grow,american history,american male,american man,american student,americanize,americans,amusement,analysis,analyst,analyze,ancestor,ancestral,ancient,andrew,andy,anecdote,angel,angeles,anger,angle,angry,anguish,animal,animation,anime,anniversary,announce,annoyed,annoying,annual,anonymous,answer,answer question,antagonize,anthony,anti,anticipation,antonio,anxiety,anxious,anybody,anymore,anymore know,anyways,apart,apartment,api,api community,api equality,apologetic,apologize,apology,app,app like,apparently,appeal,appear,appearance,apple,application,apply,appreciate,appreciation,approach,approach date,appropriate,approve,april,architect,architecture,arduous,area,area passionate,arguably,argue,argument,arise,arm,army,arrange,arrangement,arrival,arrive,art,art form,art passionate,article,articulate,artist,artistic,ashamed,ashamed sexuality,asia,asian,asian america,asian american,asian americans,asian asian,asian community,asian country,asian culture,asian date,asian ethnicity,asian friend,asian gay,asian guy,asian identity,asian male,asian man,asian pacific,asian parent,asian people,asian pride,asian student,asian white,asians,asians asian,asians date,aside,ask,ask boyfriend,ask gay,ask help,ask know,ask like,ask mother,ask question,ask want,asleep,aspect,aspiration,aspire,ass,asshole,assimilation,assistant,associate,assume,assumption,assure,atheist,attach,attain,attempt,attend,attendant,attention,attitude,attorney,attract,attraction,attractive,attractive asian,attractiveness,attribute,audience,aunt,australia,australian,authentic,authenticity,author,automatically,available,avenue,average,avid,avoid,await,awake,award,aware,awareness,away,away want,awesome,awful,awhile,awkward,ba,baby,bachelor,background,backpack,backwards,bad,bad case,bad thing,bad way,badly,badly want,bag,baggage,bake,baker,bakery,baking,balance,ball,bamboo,band,bangkok,bank,banter,bar,bare,barely,barista,barrier,base,basic,basically,basis,bathroom,battle,bay,bay area,beach,bear,bear chinese,bear raise,beat,beautiful,beautiful thing,beautifully,beauty,bed,bed night,beg,begin,begin explore,beginning,behave,behavior,beijing,belief,believe,believe deserve,believe good,believe love,believe make,believe parent,believe people,believe world,believer,belong,belonging,belt,beneath,beneficial,benefit,berkeley,bet,better,betterment,bi,bias,bible,big,big deal,big picture,big thank,bigotry,bike,bilingual,billion,binary,bind,binge,binge watch,biological,biology,birth,birthday,bisexual,bit,bit piece,bitch,bite,bitter,bitter taste,bitterness,black,black white,blah,blame,blanket,blast,blatant,blatant racism,bleed,blend,bless,blessing,blind,block,blog,blogg

In [338]:
tfidf_doc_term['come out'].sum()
tfidf_doc_term['come'].sum()
tfidf_doc_term['out'].sum()

4.434429651528141

8.98228260438847

12.04835052079319

In [282]:
cv_doc_term.sum(axis=0).sort_values(ascending=False)[:20]

people       777
like         735
out          646
time         640
love         572
make         565
want         529
know         522
just         511
feel         504
community    499
think        482
gay          481
life         473
year         448
work         434
say          427
asian        415
come         413
world        406
dtype: int64

In [283]:
tfidf_doc_term.sum(axis=0).sort_values(ascending=False)[:20]

people       14.74
love         14.03
like         13.90
time         12.34
gaysian      12.19
gay          12.13
out          12.05
community    11.38
asian        11.32
know         11.24
member       11.00
make         10.95
feel         10.85
work         10.76
want         10.58
think        10.45
job          10.22
just         10.21
world        10.09
say           9.79
dtype: float64

In [304]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(tfidf_doc_term)

In [305]:
doc_topic.shape

(376, 5)

In [306]:
doc_topic_df = pd.DataFrame(doc_topic.round(2))

In [307]:
doc_topic_df.head()

,0,1,2,3,4
0,0.10,0.01,0.02,0.04,0.08
1,0.14,0.02,0.02,0.10,0.01
2,0.09,0.03,0.19,0.00,0.05
3,0.07,0.03,0.00,0.11,0.10
4,0.00,0.01,0.04,0.06,0.41


In [310]:
topic_term = pd.DataFrame(nmf_model.components_.round(2),
                         columns = tfidf.get_feature_names())

In [311]:
topic_term.head()

,aapi,abandon,abc,ability,able,able make,able meet,able travel,able work,abroad,abs,absence,absolute,absolutely,abuse,academia,academic,accent,accept,accept love,accept open,accept sexuality,acceptable,acceptance,access,accommodate,accompany,accomplish,accomplishment,accord,account,accountability,accountable,accumulate,accurately,accuse,achieve,achievement,acknowledge,acquaintance,act,act like,act way,acting,action,action speak,active,actively,activism,activist,activity,actor,actual,actually,actually like,actually quite,actually talk,actually tell,actually want,acutely,ad,adam,adapt,add,addict,addiction,addition,additional,address,address issue,adequate,adhere,adjust,admire,admission,admit,adolescent,adopt,adore,adult,adulthood,advantage,adventure,advertising,advice,advise,advisor,advocacy,advocate,aesthetic,affair,affect,affect community,affection,affectionate,affirm,affirmation,afford,affordable,afraid,african,african american,afterlife,afternoon,afterward,age,age gay,agency,agenda,aggression,ago,agree,agree disagree,aha,ahead,aids,aim,air,airport,aisle,aka,akin,alarm,albeit,alcohol,alex,align,alive,alliance,allow,allow share,allow travel,ally,alostfish,alternate,alternative,altogether,amateur,amazing,amazing friend,ambition,america,american,american bear,american community,american culture,american family,american gay,american grow,american history,american male,american man,american student,americanize,americans,amusement,analysis,analyst,analyze,ancestor,ancestral,ancient,andrew,andy,anecdote,angel,angeles,anger,angle,angry,anguish,animal,animation,anime,anniversary,announce,annoyed,annoying,annual,anonymous,answer,answer question,antagonize,anthony,anti,anticipation,antonio,anxiety,anxious,anybody,anymore,anymore know,anyways,apart,apartment,api,api community,api equality,apologetic,apologize,apology,app,app like,apparently,appeal,appear,appearance,apple,application,apply,appreciate,appreciation,approach,approach date,appropriate,approve,april,architect,architecture,arduous,area,area passionate,arguably,argue,argument,arise,arm,army,arrange,arrangement,arrival,arrive,art,art form,art passionate,article,articulate,artist,artistic,ashamed,ashamed sexuality,asia,asian,asian america,asian american,asian americans,asian asian,asian community,asian country,asian culture,asian date,asian ethnicity,asian friend,asian gay,asian guy,asian identity,asian male,asian man,asian pacific,asian parent,asian people,asian pride,asian student,asian white,asians,asians asian,asians date,aside,ask,ask boyfriend,ask gay,ask help,ask know,ask like,ask mother,ask question,ask want,asleep,aspect,aspiration,aspire,ass,asshole,assimilation,assistant,associate,assume,assumption,assure,atheist,attach,attain,attempt,attend,attendant,attention,attitude,attorney,attract,attraction,attractive,attractive asian,attractiveness,attribute,audience,aunt,australia,australian,authentic,authenticity,author,automatically,available,avenue,average,avid,avoid,await,awake,award,aware,awareness,away,away want,awesome,awful,awhile,awkward,ba,baby,bachelor,background,backpack,backwards,bad,bad case,bad thing,bad way,badly,badly want,bag,baggage,bake,baker,bakery,baking,balance,ball,bamboo,band,bangkok,bank,banter,bar,bare,barely,barista,barrier,base,basic,basically,basis,bathroom,battle,bay,bay area,beach,bear,bear chinese,bear raise,beat,beautiful,beautiful thing,beautifully,beauty,bed,bed night,beg,begin,begin explore,beginning,behave,behavior,beijing,belief,believe,believe deserve,believe good,believe love,believe make,believe parent,believe people,believe world,believer,belong,belonging,belt,beneath,beneficial,benefit,berkeley,bet,better,betterment,bi,bias,bible,big,big deal,big picture,big thank,bigotry,bike,bilingual,billion,binary,bind,binge,binge watch,biological,biology,birth,birthday,bisexual,bit,bit piece,bitch,bite,bitter,bitter taste,bitterness,black,black white,blah,blame,blanket,blast,blatant,blatant racism,bleed,blend,bless,blessing,blind,block,blog,blogg

In [317]:
words = topic_term.columns.to_list()

In [323]:
words[-1:-10:-1]

['zone',
 'zero',
 'youtuber',
 'youtube',
 'youthful',
 'youth',
 'young year',
 'young people',
 'young gay']

In [328]:
nmf_model.components_.argsort(axis=1)[:, -1:-8:-1]

array([[2766, 3375, 2452, 1541, 3057, 4886, 3040],
       [1829, 2971, 3398, 2365, 1255, 1256, 5368],
       [ 242, 2878, 5300,  142,  244, 1035,  257],
       [3966, 1035, 3517, 2766, 4807, 1207, 4833],
       [1800, 3436, 3285, 1740, 2632,  753, 2831]])

In [333]:
# Finding the top 7 words with the highest weights for each topic
indices_top_words = nmf_model.components_.argsort(axis=1)[:, -1:-8:-1]
top_topic_words = [[words[index] for index in indices_list] for indices_list in indices_top_words]

In [334]:
top_topic_words  

[['love', 'parent', 'know', 'family', 'mother', 'time', 'mom'],
 ['gaysian', 'member', 'passionate', 'job', 'dream', 'dream job', 'world'],
 ['asian', 'man', 'white', 'american', 'asian american', 'date', 'asian man'],
 ['relationship', 'date', 'person', 'love', 'thing', 'distance', 'think'],
 ['gay', 'people', 'out', 'friend', 'like', 'come', 'make']]